In [1]:
#from parse_kd import *
import re
import pandas as pd
import numpy as np
from chunk_utils import *
import matplotlib
import scipy.cluster.hierarchy as hc
import matplotlib.pyplot as plt
from pyclustering.cluster.xmeans import xmeans
data_dir = './data/dw_freetext/'

In [2]:
%load_ext autoreload
%autoreload 2
matplotlib.rcParams['figure.figsize'] = 8,5

In [3]:
df = pd.read_csv(data_dir+'GunControl_400.csv', delimiter='\t', usecols=[0,6,10])
df_filtered = df[df['Task']=='Copy_2'].reset_index(drop=True)
# "Copy_2" column = when they're stating true opinion, repeating (transcribing) the essay already written
# resrt_index() is essential!

new_uids = pd.Series(map(lambda x: "user"+str(x), range(df_filtered.shape[0])))
df_filtered = df_filtered.assign(uid=new_uids)

df_filtered.head(7)

,UserName,Task,ReviewMeta,uid
0,A10E8QHPV30BAD,Copy_2,0 MouseUp 0 0;874 KeyDown 16;946 KeyDown 73;10...,user0
1,A11T1LGIWPM9Z1,Copy_2,0 MouseUp 0 0;784 KeyDown 16;863 KeyDown 84;92...,user1
2,A145PJ769UGAID,Copy_2,0 MouseUp 0 0;1517 KeyDown 16;1517 KeyDown 87;...,user2
3,A161NZP385W1TX,Copy_2,0 MouseUp 0 0;1621 KeyDown 16;1734 KeyDown 84;...,user3
4,A168RQT8B18FH,Copy_2,0 MouseUp 0 0;2112 KeyDown 16;2311 KeyDown 73;...,user4
5,A16HFBY06FP1RJ,Copy_2,0 MouseUp 0 0;317 KeyDown 16;543 KeyDown 71;62...,user5
6,A191P01GHJRPWO,Copy_2,0 MouseUp 0 0;7007 KeyDown 16;7018 KeyDown 73;...,user6


In [4]:
phrases = ["criminal", "constitution", "amendment", "fundamental", "right", "gun", "control", 
           "safe", "citizen", "kill", "violence", "strongly", "think", "feel", 
           "legal", "reason", "fear", "oppose", "government", "power", 
           "freedom", "people"
          ]

# dict of dicts -- examining at user level
userwise_delays = {}
# dict of lists --- examining at population level (throwing away user information)
phrasewise_delays = {x:[] for x in phrases}

#for textIndex in range(df_filtered.shape[0]):
for textIndex in range(80,130):    
    rawkd = df_filtered['ReviewMeta'].iloc[textIndex]
    uid = df_filtered['uid'].iloc[textIndex]
    userwise_delays[uid] = {}
    
    for phrase in phrases:
        phrase_keys = get_phrases_from_rawkd(rawkd, phrase)    
        all_delays = [keypress2delays(phrase, keys) for keys in phrase_keys]
        userwise_delays[uid][phrase] = all_delays
        phrasewise_delays[phrase].extend(all_delays)

<b> 

pick a word

represent all users by ONE vector, the average delay vector of that word

cluster users together

<b>

In [6]:
#clusters = 
for word_of_interest in phrases:
    userwise_avgs = {}
    print ("\n\n"+word_of_interest)
    for uid in userwise_delays:
        if len(userwise_delays[uid][word_of_interest]):
            word1_avg = np.cumsum(np.average(userwise_delays[uid][word_of_interest], axis=0))
            userwise_avgs[uid] = word1_avg
    uid_keys = list(userwise_avgs.keys())
    if len(uid_keys):
        vals = np.array(list(userwise_avgs.values()))
        clus = xmeans(vals)
        clus.process()
        clusterids = clus.get_clusters()
        for i in range(len(clusterids)):
            cluster_n_uids = [uid_keys[x] for x in clusterids[i]]
            print (cluster_n_uids, " ---> cluster ", i)



criminal
['user81', 'user90', 'user100', 'user115', 'user121', 'user126']  ---> cluster  0
['user91', 'user94', 'user97', 'user102', 'user107', 'user108', 'user109', 'user114']  ---> cluster  1
['user83']  ---> cluster  2
['user120']  ---> cluster  3


constitution
['user80', 'user82', 'user96', 'user103']  ---> cluster  0
['user97']  ---> cluster  1
['user102']  ---> cluster  2
['user90', 'user105', 'user116']  ---> cluster  3


amendment
['user96', 'user97', 'user103', 'user110', 'user118']  ---> cluster  0


fundamental


right
['user80', 'user83', 'user87', 'user88', 'user90', 'user100', 'user104', 'user105', 'user116', 'user123', 'user125']  ---> cluster  0
['user84', 'user93', 'user94', 'user96', 'user97', 'user102', 'user103', 'user107', 'user109', 'user110', 'user112', 'user113', 'user117', 'user118']  ---> cluster  1


gun
['user80', 'user81', 'user82', 'user83', 'user84', 'user85', 'user86', 'user87', 'user88', 'user89', 'user90', 'user91', 'user92', 'user93', 'user94', 'us